In [6]:
#coding:utf-8
import json
import os

def readFile(read_dath_path):
    with open(read_dath_path,"r",encoding="utf-8") as fp:
        lines=fp.readlines()
    return lines

def writeFile(write_file_path,str):
    with open(write_file_path,"a",encoding="utf-8") as fp:
        fp.write(str+"\n")

def obtainFinishFile(record_file_path,write_recode_file_path):
    lines = readFile(record_file_path)
    for line in lines:
        if "完成的文件有" in line:
            line = line.strip().split("：")[1].strip()
            try:
             writeFile(write_recode_file_path,line)
            except UnicodeDecodeError as u:
                print(u)

def readJsonFile(read_json_file_path):
    with open(read_json_file_path,"r") as fp_r_json:
        json_data =json.load(fp_r_json)
    return json_data

def compareNameCategory4_2And5_0(read_name_category_42_path, read_name_category_50_path):
    n_c_4_2_dict = readJsonFile(read_name_category_42_path)
    n_c_5_0_dict = readJsonFile(read_name_category_50_path)
    n_c_4_2_set = set()
    n_c_5_0_set = set()
    stat_4_2_category_dict = {}
    stat_5_0_category_dict = {}
    stat_change_dict = {}
    stat_cate_different = 0
    stat_reduce_entity_num = 0
    stat_add_entity_num = 0
    stat_add_entity_list = set()
    
    for key_4,value_4 in n_c_4_2_dict.items():
        n_c_4_2_set.add((key_4,value_4))
        if value_4 not in stat_4_2_category_dict:
            stat_4_2_category_dict[value_4] = 1
        else:
            stat_4_2_category_dict[value_4] += 1

        if key_4 in n_c_5_0_dict and n_c_4_2_dict[key_4] != n_c_5_0_dict[key_4]:
            stat_cate_different += 1
        elif key_4 not in n_c_5_0_dict :
            stat_reduce_entity_num += 1


    for key_5,value_5 in n_c_5_0_dict.items():
        n_c_5_0_set.add((key_5,value_5))
        if value_5 not in stat_5_0_category_dict:
            stat_5_0_category_dict[value_5] = 1
        else:
            stat_5_0_category_dict[value_5] += 1

        if key_5 not in  n_c_4_2_dict:
            stat_add_entity_list.add(key_5)
            stat_add_entity_num += 1


    print("n_c_4_2_set = ", len(n_c_4_2_set))
    print("n_c_5_0_set = ", len(n_c_5_0_set))
    itersection = n_c_4_2_set & n_c_5_0_set     #  4.2 与 5.0 的交集
    print("itersection =",len(itersection))
    difference_set = n_c_4_2_set - n_c_5_0_set     # 4.2 与 5.0 的差集
    print("difference set",len(difference_set))

    print("stat 4.2 category =", stat_4_2_category_dict)
    print("stat 5.0 category =", stat_5_0_category_dict)
    for key in stat_5_0_category_dict.keys():
        if key in stat_4_2_category_dict:
            stat_change_dict[key] = stat_5_0_category_dict[key] - stat_4_2_category_dict[key]
        else:
            stat_change_dict[key] = stat_5_0_category_dict[key]
    print("stat category change = ",stat_change_dict)

    print("stat_cate_different =",stat_cate_different)
    print("stat_reduce_entity_num = ",stat_reduce_entity_num)
    print("stat_add_entity_num = ", stat_add_entity_num)
#    print("stat_add_entity_list:", stat_add_entity_list)
def readLineFile(read_dath_path):
    with open(read_dath_path,"r",encoding="utf-8") as fp:
        lines=fp.read().splitlines()
    return lines

def compare_entity_dict_and_rel_dict(read_name_category_50_path,rel_dict_dir):
    n_c_5_0_dict = readJsonFile(read_name_category_50_path)
    n_c_5_0_set = set()
    stat_different_number = 0
    remove_repeat_set = set()

    for key_5,value_5 in n_c_5_0_dict.items():
        n_c_5_0_set.add((key_5,value_5))

    rel_dict_file_list = os.listdir(rel_dict_dir)
    for rel_dict_file_name in rel_dict_file_list:
        rel_file_path = os.path.join(rel_dict_dir,rel_dict_file_name)
        lines = readLineFile(rel_file_path)
        c1,c2 = rel_dict_file_name.split("_")[:-1]
        for line in lines:
            line = json.loads(line)
            e1 = line["e1"]
            e2 = line["e2"]
            if (e1,c1) not in n_c_5_0_set and (e1,c1) not in remove_repeat_set :
                stat_different_number += 1
                remove_repeat_set.add((e1,c1))
            elif (e2,c2) not in n_c_5_0_set and (e2,c2) not in remove_repeat_set :
                stat_different_number += 1
                remove_repeat_set.add((e2,c2))


    print("not align num = ",stat_different_number)


if __name__ == "__main__":
    read_name_category_42_path = "./medical_entity_Name_Catetgory_v4.2.json"
    read_name_category_50_path = "./medical_entity_Name_Catetgory_v5.0.json"
    compareNameCategory4_2And5_0(read_name_category_42_path, read_name_category_50_path)
    #rel_dict_dir  = "./data/quchong_relation_dict/"
    #compare_entity_dict_and_rel_dict(read_name_category_50_path, rel_dict_dir)


n_c_4_2_set =  232403
n_c_5_0_set =  304081
itersection = 225961
difference set 6442
stat 4.2 category = {'disease': 40963, 'symptom': 1829, 'drugs': 28745, 'anatomy': 2246, 'phenomenon': 1678, 'equipment': 2791, 'organism': 3697, 'gene': 43840, 'compound': 79449, 'protein': 27165}
stat 5.0 category = {'disease': 53303, 'anatomy': 2272, 'symptom': 1827, 'drugs': 24515, 'phenomenon': 1678, 'equipment': 2787, 'organism': 3695, 'gene': 96587, 'compound': 91240, 'protein': 26177}
stat category change =  {'disease': 12340, 'anatomy': 26, 'symptom': -2, 'drugs': -4230, 'phenomenon': 0, 'equipment': -4, 'organism': -2, 'gene': 52747, 'compound': 11791, 'protein': -988}
stat_cate_different = 2512
stat_reduce_entity_num =  3930
stat_add_entity_num =  75608
stat_add_entity_list: {'absence of intrinsic factor', 'perfluorobutanesulfonic acid', 'a-beta', 'mandibular condyle aplasia', 'slc37a4b', 'at3g28450', 'at5g17850', 'rpa14', 'be10.1', 'd1014.2', 'cg13711', 'cdc53', "primary sj枚gren's syndrome"

In [5]:
print(stat_add_entity_list)

NameError: name 'stat_add_entity_list' is not defined